## Logistic Regression Model

In [4]:
# Initial imports.
import pandas as pd
import numpy as np
from path import Path
from pathlib import Path
from collections import Counter
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
# load data
file_path = Path('./Resources/ml_clean_stroke_dataset.csv')
stroke_df = pd.read_csv(file_path)
stroke_df.head()

,Age,Gender,Ever Married,Smoking Status,Hypertension,Heart Disease,Stroke
0,3,Male,No,unknown,0,0,0
1,58,Male,Yes,never smoked,1,0,0
2,8,Female,No,unknown,0,0,0
3,70,Female,Yes,formerly smoked,0,0,0
4,14,Male,No,unknown,0,0,0


## Preprocess data

In [10]:
# Use scikit-learn to encode data
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

stroke_df['Gender'] = le.fit_transform(stroke_df['Gender'])
stroke_df['Ever Married'] = le.fit_transform(stroke_df['Ever Married'])
stroke_df['Smoking Status'] = le.fit_transform(stroke_df['Smoking Status'])
stroke_df.head()

,Age,Gender,Ever Married,Smoking Status,Hypertension,Heart Disease,Stroke
0,3,1,0,3,0,0,0
1,58,1,1,1,1,0,0
2,8,0,0,3,0,0,0
3,70,0,1,0,0,0,0
4,14,1,0,3,0,0,0


## Scaling & Normalization

In [11]:
# load scaler
data_scaler = StandardScaler()

In [12]:
# train the scaler and transform the data
stroke_data_scaled = data_scaler.fit_transform(stroke_df)

In [13]:
# preview scaled data
stroke_data_scaled[:5]

array([[-1.73913268,  1.20108119, -1.34420288,  1.28640708, -0.32129564,
        -0.22334159, -0.13554685],
       [ 0.70066001,  1.20108119,  0.74393532, -0.53968634,  3.11239826,
        -0.22334159, -0.13554685],
       [-1.51733334, -0.83084068, -1.34420288,  1.28640708, -0.32129564,
        -0.22334159, -0.13554685],
       [ 1.23297842, -0.83084068,  0.74393532, -1.45273305, -0.32129564,
        -0.22334159, -0.13554685],
       [-1.25117414,  1.20108119, -1.34420288,  1.28640708, -0.32129564,
        -0.22334159, -0.13554685]])

In [14]:
# verify the mean
print(np.mean(stroke_data_scaled[:,0]))
print(np.std(stroke_data_scaled[:,0]))

-1.0314330035227261e-16
0.9999999999999999


####  Standardization was  successful, and all numerical columns now have a mean of 0 and a standard deviation of 1, reducing the likelihood that large values will unduly influence the model.

## Split Data into Training and Testing

In [15]:
# Create our features
X = pd.get_dummies(stroke_df.drop(columns="Stroke"))

# Create our target
y= stroke_df["Stroke"]

In [16]:
# Check the balance of our target values
y.value_counts()

0    42617
1      783
Name: Stroke, dtype: int64

In [17]:
# Split into training and testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [18]:
X_train.shape

(32550, 6)

In [19]:
X_test.shape

(10850, 6)

## Oversample Data

In [20]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

Counter(y_resampled)

Counter({0: 31950, 1: 31950})

## Run Logistic Regression

In [21]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [22]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[7919, 2748],
       [  47,  136]])

In [27]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,7919,2748
Actual 1,47,136


## Results 
#### - Out of 10,667 no stroke outcomes (Actual 0), 7,919 were predicted to be no stroke (Predicted 0), which are true positives. 
#### - Out of 10,667 no stroke outcomes (Actual 0), 2,748 were predicted to suffer from a stroke (Predicted 1), which are considered false negatives.
#### - Out of 183 stroke outcomes (Actual 1), 47 were predicted to be no stroke (Predicted 0) and are considered false positives.
#### - Out of 183  stroke outcomes (Actual 1), 136 were predicted to be strokes (Predicted 1) and are considered true negatives.
#### - The model was correct 74.2% of the time.

In [24]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.7427762247183873

In [26]:
# check accuracy of model
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.7423963133640553


In [25]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.99      0.74      0.74      0.85      0.74      0.55     10667
          1       0.05      0.74      0.74      0.09      0.74      0.55       183

avg / total       0.98      0.74      0.74      0.84      0.74      0.55     10850

